In [1]:
import numpy as np

In [29]:
positives = np.load('../stacked_data/numpy_transformed/positive_seq_and_score.npy')
negatives = np.load('../stacked_data/numpy_transformed/negative_seq_and_score.npy')

In [30]:
positives[~np.isfinite(positives)] = 0
negatives[~np.isfinite(negatives)] = 0

In [31]:
positives.shape, negatives.shape

((186912, 134), (186912, 134))

In [32]:
print(np.isnan(positives).any())
print(np.isnan(negatives).any())
print(np.isfinite(negatives).all())
print(np.isfinite(positives).all())

False
False
True
True


In [18]:
#normalize:
def norm(x):
  sums = x.sum(axis=1)
  return x/sums[:, np.newaxis]
positives = norm(positives)
negatives = norm(negatives)

/Users/nguyen/anaconda3/envs/TreeNLG/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [33]:
import torch
from torch import nn

In [34]:
class DoubleHead(nn.Module):
  def __init__(self):
    super(DoubleHead, self).__init__()
    self.lstm = nn.LSTM(1, 64)
    self.dense_1 = nn.Linear(64, 32)
    self.dense_2 = nn.Linear(1, 1)
    self.dense_3 = nn.Linear(33, 1)
    self.relu = nn.ReLU()

  def forward(self, input_1, input_2):
    out, _ = self.lstm(input_1)
    out = self.relu(out)
    out = self.dense_1(out)
    out = self.relu(out)
    out2 = self.dense_2(input_2)
    out = torch.sum(out, dim=1)
    out2 = self.relu(out2)
    combined = torch.cat((out.view(out.size(0), -1),
                          out2.view(out2.size(0), -1)), dim=1)
    out = self.dense_3(combined)
    return out
    

In [35]:
targets = np.ones((positives.shape[0], 1))
pos_in_1, pos_in_2 = np.hsplit(positives, [-1])
neg_in_1, neg_in_2 = np.hsplit(negatives, [-1])

In [36]:
pos_in_1 = torch.from_numpy(pos_in_1).float()
pos_in_2 = torch.from_numpy(pos_in_2).float()
neg_in_1 = torch.from_numpy(neg_in_1).float()
neg_in_2 = torch.from_numpy(neg_in_2).float()
targets = torch.from_numpy(targets).float()

In [37]:
pos_in_1 = pos_in_1.unsqueeze(-1)
neg_in_1 = neg_in_1.unsqueeze(-1)

In [43]:
model = DoubleHead()

In [44]:
loss = torch.nn.BCELoss()
opt = torch.optim.SGD(model.parameters(), lr=0.001)

In [45]:
batch_size = 128
sig = nn.Sigmoid()
for i in range(100):
  permutation = torch.randperm(targets.size()[0])
  for j in range(0, targets.size()[0], batch_size):
    opt.zero_grad()
    indices = permutation[j:j+batch_size]
    pos_batch_in_1, pos_batch_in_2 = pos_in_1[indices], pos_in_2[indices]
    neg_batch_in_1, neg_batch_in_2 = neg_in_1[indices], neg_in_2[indices]
    target_batch = targets[indices]
    pos_out = model(pos_batch_in_1, pos_batch_in_2)
    neg_out = model(neg_batch_in_1, neg_batch_in_2)
    margin = sig(pos_out - neg_out)
    epoch_loss = loss(margin, target_batch)
    epoch_loss.backward()
    opt.step()
  if i % 10 == 0:
      print(i, epoch_loss.item())

0 0.6896109580993652
10 0.6957273483276367
20 0.5017055869102478
30 0.16276729106903076
40 0.09231386333703995
50 0.07377491146326065
60 0.048929788172245026
70 0.03806595504283905
80 1.1033697128295898
90 0.08307085931301117


In [42]:
old = model

In [70]:
a, b = neg_in_1[0], neg_in_2[0]

In [71]:
a = torch.from_numpy(np.expand_dims(a, 0))

In [72]:
a.shape, b.shape

(torch.Size([1, 133, 1]), torch.Size([1]))

In [78]:
model(a, b).item()

-141.86825561523438

In [75]:
!ls ../

CODE_OF_CONDUCT.md   compute_tree_acc.py  optim
CONTRIBUTING.md      constrained_decoding options.py
LICENSE              data                 scripts
README.md            data-prep            stacked_data
__init__.py          e2e-metrics          tasks
__pycache__          example
checkpoints          notebooks


In [76]:
torch.save(model, '../ranker.pt')

/Users/nguyen/anaconda3/envs/TreeNLG/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type DoubleHead. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [77]:
torch.save(model.state_dict(), '../ranker_state.pts')